In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [ ]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20201018044700.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000/",        "nid": 3}}

env = connections[network]

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
contracts = {}

# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
    'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'},
  'staking': {'zip': 'core_contracts/staking.zip',
    'SCORE': 'cx127ab2bd9face92d02c78fff9bc1956657ac2661'},
  'sicx': {'zip': 'token_contracts/sicx.zip',
    'SCORE': 'cxec4ef61076fb216c12de32965eee0f9a1c36496a'},
  'icd': {'zip': 'token_contracts/icd.zip',
    'SCORE': 'cxe510ed4ae3889d3e289b097226758446774ddbcc'},
  'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
    'SCORE': 'cx6a4d643725696fc964874bbb2c7fc56e98651d2d'},
  'bal': {'zip': 'token_contracts/bal.zip',
    'SCORE': 'cx5393c47e17123eae5fdda35ba458a31f82127477'}}

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [ ]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
deploy = ['loans', 'staking', 'sicx', 'icd', 'dummy_oracle', 'bal']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

## Deploy All SCOREs

In [ ]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
contracts

### Deploy a single SCORE

In [ ]:
# Deploy a single SCORE
score = 'loans'
loc = {'zip': 'core_contracts/loans.zip',  'SCORE': ''}
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(loc["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')
res

### Importable methods test module

In [ ]:
api = icon_service.get_score_api(contracts['loans']['SCORE'])

In [ ]:
api

In [ ]:
module = ("from iconsdk.icon_service import IconService\n"
        + "from iconsdk.providers.http_provider import HTTPProvider\n"
        + "from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder\n"
        + "from iconsdk.builder.call_builder import CallBuilder\n"
        + "from iconsdk.signed_transaction import SignedTransaction\n"
        + "icon_service = IconService(HTTPProvider('http://52.53.175.151:9000/', 3))\n"
        + "\n"
        + "class Test(object):\n"
        + "\n"
        + "    def __init__(self):\n"
        + "        pass\n"
        + "\n")
filename = 'methods.py'
for item in api:
    if item.get('readonly', '') == '0x1' and item['type'] == 'function':
        func_params = [f"{param['name']}" for param in item['inputs']]
        params = "{" + ", ".join([f"\'{param}\': {param}" for param in func_params]) + "}"
        module += (f"    def {item['name']}({', '.join(['self'] + func_params)}):\n"
                 + f"        call = CallBuilder().from_('{wallet.get_address()}')\\\n"
                 + f"            .to('{contracts['loans']['SCORE']}')\\\n"
                 + f"            .method('{item['name']}')\\\n"
                 + f"            .params({params})\\\n"
                 + f"            .build()\n"
                 + f"        return icon_service.call(call)\n"
                 + f"\n")
with open(filename, 'w') as f:
    f.write(module)

from methods import Test

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
tests = Test()

In [ ]:
tests.name()
tests.asset_count()
int(tests.get_total_collateral(), 0) / ICX
tests.get_available_assets()
tests.get_account_positions(wallet.get_address())

## Update a SCORE

In [ ]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


## Set parameters

In [ ]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'icd', 'method': 'set_oracle_address', 'params':{'_address': contracts['dummy_oracle']['SCORE']}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
#             {'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
#             {'contract': 'sicx', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'staking', 'method': 'set_sICX_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE']}},
            {'contract': 'sicx', 'method': 'set_staking_address', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]

In [ ]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

## Run through read-only methods on all SCOREs

### A single read-only method

In [ ]:
params = {'_owner': wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_account_positions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
ro_methods = {"loans": ["name", "get_available_assets", "asset_count", "get_total_collateral"],
              "staking": ["name", "get_sICX_address", "get_rate"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [ ]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result

## Deposit ICX and Mint ICD Loan

In [ ]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Just deposit collateral

In [ ]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Repay Some ICD

In [ ]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Withdraw Some Collateral

In [ ]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdraw_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))